In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)
# Pickle package
import pickle

In [2]:
# import the data
data = pd.read_csv("clean_data.csv")
print(data.head())

   Unnamed: 0                                               body  \
0           0  يبغى التنبيه على ان السعوديه  تستخدم صواريخ جو...   
1           1  امريكا قتلت بالامس معوق رفض رفع يديه فماذا تري...   
2           2  هذا الشخص هو من كان مؤيد لاحتلال العراق وضرب ا...   
3           3  الى جمال ريان مذيع الجزيره  نحن من رعاك فى الم...   
4           4  خيبه  الامل ليست تشاؤما ولا تقولا    ٠   عزم ل...   

   languagecomment  
0               -1  
1               -1  
2               -1  
3               -1  
4                0  


In [18]:
# # Plot two ingredients
# sns.lmplot('body', 'languagecomment', data=data, hue='Type',
#            palette='Set1', fit_reg=False, scatter_kws={"s": 70});

In [19]:
label_data = data[['body', 'languagecomment']]
label_data[:10]

,body,languagecomment
0,يبغى التنبيه على ان السعوديه تستخدم صواريخ جو...,-1
1,امريكا قتلت بالامس معوق رفض رفع يديه فماذا تري...,-1
2,هذا الشخص هو من كان مؤيد لاحتلال العراق وضرب ا...,-1
3,الى جمال ريان مذيع الجزيره نحن من رعاك فى الم...,-1
4,خيبه الامل ليست تشاؤما ولا تقولا ٠ عزم ل...,0
5,سفينه الاكاذيب والنفاق والفاسد والاستبداد,-1
6,لست ادرى من هو الذى ستنطفىء شمعته وتتحرق م...,-1
7,جمال خاشقجى اللبرالى العلمانى يتحول من داعيه ع...,-1
8,طرف ثالث محايد مثل من ؟ الامم المتحده ...,-1
9,"عجيب , لقد اصبحت تعريف كلمه الارهاب بما ...",-1


In [20]:
import keras.backend as K
from keras import Input, Model
from keras.initializers import glorot_normal, orthogonal
from keras.layers import Embedding, SpatialDropout1D, Bidirectional, GRU, Flatten, Dense, Dropout, BatchNormalization, \
    Reshape, Conv2D, MaxPool2D, Concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, LSTM
from keras.optimizers import Adam

from algo.nn.layers import Capsule, Attention
from algo.nn.wrappers import DropConnect


def capsule(maxlen, max_features, embed_size, embedding_matrix):
    K.clear_session()
    inp = Input(shape=(maxlen,))
    #     word = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    #     emoji = Embedding(max_features, embed_size, weights=[emoji_embedding_matrix], trainable=False)(inp)
    #     x = concatenate([(word), (emoji)])

    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)

    x = SpatialDropout1D(rate=0.2)(x)
    #     x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Bidirectional(GRU(100, return_sequences=True,
                          kernel_initializer=glorot_normal(seed=12300),
                          recurrent_initializer=orthogonal(gain=1.0, seed=10000)))(x)

    x = Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
    x = Flatten()(x)

    x = Dense(100, activation="relu", kernel_initializer=glorot_normal(seed=12300))(x)
    x = Dropout(0.12)(x)
    x = BatchNormalization()(x)

    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model




Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

ModuleNotFoundError: No module named 'algo'

In [21]:

def cnn_2d(maxlen, max_features, embed_size, embedding_matrix):
    K.clear_session()

    filter_sizes = [1, 2, 3, 5]
    num_filters = 32

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.4)(x)
    x = Reshape((maxlen, embed_size, 1))(x)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), kernel_initializer='normal',
                    activation='elu')(x)
    conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embed_size), kernel_initializer='normal',
                    activation='elu')(x)

    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
    maxpool_3 = MaxPool2D(pool_size=(maxlen - filter_sizes[3] + 1, 1))(conv_3)

    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


def pooled_gru(maxlen, max_features, embed_size, embedding_matrix):
    K.clear_session()
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(1, activation="sigmoid")(conc)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


def lstm_attention(maxlen, max_features, embed_size, embedding_matrix):
    K.clear_session()
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(256, activation="relu")(x)
    # x = Dropout(0.25)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


def lstm_gru_attention(maxlen, max_features, embed_size, embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(LSTM(40, return_sequences=True))(x)
    y = Bidirectional(GRU(40, return_sequences=True))(x)

    atten_1 = Attention(maxlen)(x)  # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)

    conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


def attention_capsule(maxlen, max_features, embed_size, embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(rate=0.24)(x)
    x = Bidirectional(LSTM(80,
                           return_sequences=True,
                           kernel_initializer=glorot_normal(seed=1029),
                           recurrent_initializer=orthogonal(gain=1.0, seed=1029)))(x)

    x_1 = Attention(maxlen)(x)
    x_1 = DropConnect(Dense(32, activation="relu"), prob=0.2)(x_1)

    x_2 = Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
    x_2 = Flatten()(x_2)
    x_2 = DropConnect(Dense(32, activation="relu"), prob=0.2)(x_2)
    conc = concatenate([x_1, x_2])
    # conc = add([x_1, x_2])
    outp = Dense(1, activation="sigmoid")(conc)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [14]:
from sklearn.model_selection import train_test_split
from scipy import stats

train, test = train_test_split(label_data, test_size=0.2)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn import svm
from sklearn.svm import SVC, LinearSVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, recall_score, precision_score
from time import time

max_features = 5000

x = train['body']
y = train['languagecomment']


tfidf = TfidfVectorizer(max_features=max_features)

pipeline = Pipeline([
    ('tfidf', tfidf),
    (LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)) # other parameters:
    # SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
    # svm.SVC(kernel='rbf')
    # svm.SVC(kernel='linear')
    
parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2')   
}

grid_search = GridSearchCV(pipeline, parameters, cv=5,
                               n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
print(parameters)
t0 = time()
grid_search.fit(x, y)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
#refitting on entire training data using best settings
grid_search.refit

x_test = test['body']
y_test = test['languagecomment']

predicted = grid_search.predict(x_test)
print("accuracy=", np.mean(predicted == y_test))
print("recall=", recall_score(y_test, predicted, average='weighted') )
print("precision=", precision_score(y_test, predicted, average='weighted'))
print("weighted f-score", f1_score(y_test, predicted, average='weighted'))
print("Confusion matrix\n", confusion_matrix(y_test, predicted))

SyntaxError: invalid syntax (<ipython-input-15-c5743c57d73d>, line 33)